### 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

loader = PyPDFLoader('../data/콘텐츠분쟁해결_사례.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

  총 109페이지 로드 완료


In [4]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        #  법률 사례는 1200-1800자 권장
    chunk_overlap=300,      #  사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",     #  법률 문서 섹션 구분자
        "\n【쟁점사항】",     #  쟁점 부분 구분
        "\n【처리경위】",     #  처리 과정 구분
        "\n【처리결과】",     #  결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
chunks = splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  #  한국어 법률 용어에 최적
    dimensions=1536                  #  고성능 임베딩 차원
)

vectorstore = FAISS.from_documents(chunks, embeddings)
print(f" FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")

retriever = vectorstore.as_retriever(
    search_type="similarity",        #  또는 "mmr" (다양성 고려시)
    search_kwargs={"k": 5}          #  상위 5개 관련 사례 검색
)

llm = ChatOpenAI(
    model="gpt-4o",                 #  또는 "gpt-4o-mini" (경제적)
    temperature=0.2,                #  법률 조언은 정확성 중시
    max_tokens=2000                 #  충분한 답변 길이
)

prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요               #  실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

전문 법률 조언:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                        #  위에서 설정한 LLM
    chain_type="stuff",             #  문서들을 하나로 합쳐서 처리
    retriever=retriever,            #  위에서 설정한 검색기
    chain_type_kwargs={"prompt": prompt},  #  법률 자문 프롬프트
    return_source_documents=True    #  참조 문서도 함께 반환
)

test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"


for i, question in enumerate(test_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")
    
    # RAG 실행
    result = qa_chain.invoke({"query": question})
    qtype = classify_dispute_type(question)
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(qtype)
    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)


 FAISS 벡터스토어 생성 완료 (104개 벡터)

【테스트 1/5】
 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
 답변 생성 중...

 답변:
--------------------------------------------------
게임
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 다음과 같이 드립니다.

1. **사례 기반 답변**:
   - 제시된 사례 1-가-1(2006)에서는 시스템 오류로 인해 아이템이 소멸된 경우, 계정 명의자가 아닌 신청인의 복구 요청이 거부되었습니다. 이는 계정 명의자만이 복구 요청을 할 수 있다는 게임사의 약관에 근거한 결정입니다.
   - 사례 1-가-1(2009)에서는 시스템 오류로 인한 게임머니 소실 주장이 있었으나, 시스템 오류가 발생하지 않았다는 게임사의 판단에 따라 복구가 거부되었습니다.

2. **법적 근거 제시**:
   - 일반적으로 게임사의 약관은 계약의 일부분으로 간주되며, 약관에 명시된 내용에 따라 복구 여부가 결정됩니다. 따라서 약관의 내용을 확인하는 것이 중요합니다.
   - 민법 제250조와 제251조는 도난 또는 유실물에 대한 반환 청구에 관한 규정을 두고 있으나, 이는 주로 물리적 재화에 대한 규정입니다. 디지털 아이템의 경우, 게임사의 약관과 정책이 우선 적용될 수 있습니다.

3. **판례 참조**:
   - 사례 1-가-1(2008)에서는 보안 서비스 미작동으로 인한 아이템 도난이 인정되어 복구가 이루어졌습니다. 이는 게임사의 서비스 미작동이 확인된 경우에 해당합니다.

4. **실무 가이드**:
   - **약관 확인**: 게임사의 약관을 확인하여 시스템 오류로 인한 아이템 소실에 대한 복구 정책을 이해합니다.
   - **증거 수집**: 시스템 오류가 발생한 시점의 스크린샷, 로그 파일 등 가능한 모든 증거를 수집합니다.
   - **공식